In [1]:
import pandas as pd
import numpy as np
import warnings
# Disable warnings
warnings.filterwarnings("ignore")

## Extracting features of 2018 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [3]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [4]:
df

Opening  Opening.1                    Title  \
0      J A N U A R Y          5  Insidious: The Last Key   
1      J A N U A R Y          5         The Strange Ones   
2      J A N U A R Y         12             The Commuter   
3      J A N U A R Y         12               Proud Mary   
4      J A N U A R Y         12         Acts of Violence   
..               ...        ...                      ...   
243  D E C E M B E R         21               Second Act   
244  D E C E M B E R         25          Holmes & Watson   
245  D E C E M B E R         25                     Vice   
246  D E C E M B E R         25      On the Basis of Sex   
247  D E C E M B E R         25                Destroyer   

                                    Production company  \
0    Universal Pictures / Blumhouse Productions / S...   
1                               Vertical Entertainment   
2        Lionsgate / StudioCanal / The Picture Company   
3                                          Screen Gems   
4                                   Lionsgate Premiere   
..                                                 ...   
243                                  STX Entertainment   
244  Columbia Pictures / Gary Sanchez Productions /...   
245          Annapurna Pictures / Plan B Entertainment   
246                                     Focus Features   
247                                 Annapurna Pictures   

                                         Cast and crew   Ref.  
0    Adam Robitel (director); Leigh Whannell (scree...    [2]  
1    Christopher Radcliff (director/screenplay); La...    [3]  
2    Jaume Collet-Serra (director); Byron Willinger...    [4]  
3    Babak Najafi (director); John S. Newman, Chris...    [5]  
4    Brett Donowho (director); Nicolas Aaron Mezzan...    [6]  
..                                                 ...    ...  
243  Peter Segal (director); Justin Zackham, Elaine...  [237]  
244  Etan Cohen (director/screenplay); Will Ferrell...  [142]  
245  Adam McKay (director/screenplay); Christian Ba...  [117]  
246  Mimi Leder (director); Daniel Stiepleman (scre...  [207]  
247  Karyn Kusama (director); Phil Hay, Matt Manfre...  [238]  

[248 rows x 6 columns]

In [5]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'bd9d93e4649f8d7b42731beda117f8c6'
from tmdbv3api import Movie
tmdb_movie = Movie()
from tqdm import tqdm
tqdm.pandas()

In [6]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " "
            for i in range(len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN

# Modify the function to print the index and title, and handle the error
def safe_get_genre(title, index):
    try:
        return get_genre(title)
    except Exception as e:
        print(f"Error processing index: {index}")
        print(f"Title: {title}")
        print(f"Error: {e}")
        return np.NaN

# Apply the function with progress bar
df['genres'] = df.progress_apply(lambda row: safe_get_genre(str(row['Title']), row.name), axis=1)

 77%|█████████████████████████████████████████████████████████████▎                  | 190/248 [04:29<00:20,  2.89it/s]

Error processing index: 141
Title: The Miseducation of Cameron Post
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Read timed out. (read timeout=None)
Error processing index: 142
Title: Never Goin' Back
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=Never%20Goin%27%20Back&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7F5B5490>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinfo failed)"))
Error processing index: 143
Title: Dog Days
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=Dog%20Days&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7F5B7910>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinf

100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [04:30<00:00,  1.09s/it]

Error processing index: 232
Title: The Cabin
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=The%20Cabin&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7CE1E290>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinfo failed)"))
Error processing index: 233
Title: Mary Queen of Scots
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=Mary%20Queen%20of%20Scots&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7F57CD50>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinfo failed)"))
Error processing index: 234
Title: Ben Is Back
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/se

In [7]:
def get_id(x):
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        return movie_id

# Modify the function to print the index and title, and handle the error
def safe_get_id(title, index):
    try:
        return get_id(title)
    except Exception as e:
        print(f"Error processing index: {index}")
        print(f"Title: {title}")
        print(f"Error: {e}")
        return np.NaN

# Apply the function with progress bar
df['id'] = df.progress_apply(lambda row: safe_get_id(str(row['Title']), row.name), axis=1)

 62%|████████████████████████████████████████████████▊                             | 155/248 [00:00<00:00, 1124.27it/s]

Error processing index: 141
Title: The Miseducation of Cameron Post
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=The%20Miseducation%20of%20Cameron%20Post&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7F5B5310>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinfo failed)"))
Error processing index: 142
Title: Never Goin' Back
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=Never%20Goin%27%20Back&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7F50A490>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinfo failed)"))
Error processing index: 143
Title: Dog Days
Error: HTTPSConnectionPool(host='api.themoviedb.org', port

100%|███████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 663.02it/s]

Error processing index: 242
Title: Welcome to Marwen
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=Welcome%20to%20Marwen&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7F5FD010>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinfo failed)"))
Error processing index: 243
Title: Second Act
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=bd9d93e4649f8d7b42731beda117f8c6&query=Second%20Act&page=1&language=en-US (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001AB7F5FFD10>: Failed to resolve 'api.themoviedb.org' ([Errno 11001] getaddrinfo failed)"))
Error processing index: 244
Title: Holmes & Watson
Error: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/se

In [8]:
df

Opening  Opening.1                    Title  \
0      J A N U A R Y          5  Insidious: The Last Key   
1      J A N U A R Y          5         The Strange Ones   
2      J A N U A R Y         12             The Commuter   
3      J A N U A R Y         12               Proud Mary   
4      J A N U A R Y         12         Acts of Violence   
..               ...        ...                      ...   
243  D E C E M B E R         21               Second Act   
244  D E C E M B E R         25          Holmes & Watson   
245  D E C E M B E R         25                     Vice   
246  D E C E M B E R         25      On the Basis of Sex   
247  D E C E M B E R         25                Destroyer   

                                    Production company  \
0    Universal Pictures / Blumhouse Productions / S...   
1                               Vertical Entertainment   
2        Lionsgate / StudioCanal / The Picture Company   
3                                          Screen Gems   
4                                   Lionsgate Premiere   
..                                                 ...   
243                                  STX Entertainment   
244  Columbia Pictures / Gary Sanchez Productions /...   
245          Annapurna Pictures / Plan B Entertainment   
246                                     Focus Features   
247                                 Annapurna Pictures   

                                         Cast and crew   Ref.  \
0    Adam Robitel (director); Leigh Whannell (scree...    [2]   
1    Christopher Radcliff (director/screenplay); La...    [3]   
2    Jaume Collet-Serra (director); Byron Willinger...    [4]   
3    Babak Najafi (director); John S. Newman, Chris...    [5]   
4    Brett Donowho (director); Nicolas Aaron Mezzan...    [6]   
..                                                 ...    ...   
243  Peter Segal (director); Justin Zackham, Elaine...  [237]   
244  Etan Cohen (director/screenplay); Will Ferrell...  [142]   
245  Adam McKay (director/screenplay); Christian Ba...  [117]   
246  Mimi Leder (director); Daniel Stiepleman (scre...  [207]   
247  Karyn Kusama (director); Phil Hay, Matt Manfre...  [238]   

                      genres        id  
0            Horror Thriller  406563.0  
1              Drama Mystery  246252.0  
2    Action Thriller Mystery  399035.0  
3      Thriller Action Crime  442064.0  
4      Action Crime Thriller  479040.0  
..                       ...       ...  
243                      NaN       NaN  
244                      NaN       NaN  
245                      NaN       NaN  
246                      NaN       NaN  
247                      NaN       NaN  

[248 rows x 8 columns]

In [9]:
df_2018 = df[['Title','Cast and crew','genres','id']]

In [10]:
df_2018

Title  \
0    Insidious: The Last Key   
1           The Strange Ones   
2               The Commuter   
3                 Proud Mary   
4           Acts of Violence   
..                       ...   
243               Second Act   
244          Holmes & Watson   
245                     Vice   
246      On the Basis of Sex   
247                Destroyer   

                                         Cast and crew  \
0    Adam Robitel (director); Leigh Whannell (scree...   
1    Christopher Radcliff (director/screenplay); La...   
2    Jaume Collet-Serra (director); Byron Willinger...   
3    Babak Najafi (director); John S. Newman, Chris...   
4    Brett Donowho (director); Nicolas Aaron Mezzan...   
..                                                 ...   
243  Peter Segal (director); Justin Zackham, Elaine...   
244  Etan Cohen (director/screenplay); Will Ferrell...   
245  Adam McKay (director/screenplay); Christian Ba...   
246  Mimi Leder (director); Daniel Stiepleman (scre...   
247  Karyn Kusama (director); Phil Hay, Matt Manfre...   

                      genres        id  
0            Horror Thriller  406563.0  
1              Drama Mystery  246252.0  
2    Action Thriller Mystery  399035.0  
3      Thriller Action Crime  442064.0  
4      Action Crime Thriller  479040.0  
..                       ...       ...  
243                      NaN       NaN  
244                      NaN       NaN  
245                      NaN       NaN  
246                      NaN       NaN  
247                      NaN       NaN  

[248 rows x 4 columns]

In [11]:
def get_director(x):
    
    if " (director)" in x:
        return x.split("(director)")[0].split(";")[-1].strip()
    
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    
    elif " (director/screenplay)" in x:
        return x.split("(director/screenplay)")[0]
    
    elif " (directors/screenplay)" in x:
        return x.split("(directors/screenplay)")[0]
    
    elif "(director/screenplay/narrator)" in x:
        return x.split("(director/screenplay/narrator)")[0] 
    
    elif "(director[177]);" in x:
        return x.split("(director[177]);")[0]
    
    elif "(director/screenplay[119]);" in x:
        return x.split("(director/screenplay[119]);")[0]    
    
    else:
        return x.split("(director/​screenplay)")[0]

In [12]:
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))

In [13]:
t=0
for i in df_2018['director_name']:
    t=t+1
    print(t,i)

1 Adam Robitel
2 Lauren Wolkstein
3 Jaume Collet-Serra
4 Babak Najafi
5 Brett Donowho
6 Trudie Styler
7 Sam Hoffman 
8 Nicolai Fuglsig
9 Christian Gudegast 
10 Bethany Ashton Wolf 
11 Jesse Keller 
12 Wes Ball
13 Ben Lewin
14 The Spierig Brothers 
15 Sebastián Lelio 
16 Mario Van Peebles 
17 Julius Onah
18 Bryan Coyne 
19 Will Gluck 
20 R. Balki 
21 James Foley
22 Clint Eastwood
23 Brian Crano 
24 Alex Ross Perry 
25 Ryan Coogler 
26 Mark Pellington 
27 Bruce Macdonald
28 John Francis Daley, Jonathan Goldstein
29 Alex Garland 
30 Michael Sucsy
31 David Freyne 
32 Francis Lawrence
33 Eli Roth
34 Shawn Christensen 
35 Usher Morgan 
36 Ava DuVernay
37 Nash Edgerton
38 Cory Finley 
39 Johannes Roberts
40 Rob Cohen
41 Roar Uthaug
42 Greg Berlanti
43 Erwin brothers
44 José Padilha
45 Laurie Collyer
46 Eric England
47 Max Winkler 
48 Steven S. DeKnight 
49 Wes Anderson 
50 John Stevenson
51 Steven Soderbergh
52 Andrew Hyatt 
53 Stanley Tucci 
54 Scott Speer
55 Steven Spielberg
56 Tyler Perry 

In [14]:
def get_actor1(x):
    if " (screenplay)" in x:
        return ((x.split("(screenplay); ")[-1]).split(", ")[0]) 
    
    elif " (screenplay[219]);"in x:
        return ((x.split('(screenplay[219]); ')[-1]).split(",")[0])
    
    elif " (director)"in x:
        return ((x.split('(director); ')[-1]).split(",")[0])
    
    elif "(director/screenplay/narrator)" in x:
        return (x.split('(director/screenplay/narrator)'))[0]
    
    elif "(director/screenplay)" in x:
        return (x.split('(director/screenplay)'))[0] 
    
    elif "(directors/screenplay);" in x:
        return (x.split('(directors/screenplay); '))[-1].split(",")[0]
    
    elif "(directors/screenplay)," in x:
        return (x.split("(directors/screenplay), "))[-1].split(",")[0]
    
    else:
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[0]) 

In [15]:
df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))

In [16]:
t=0
for i in df_2018['actor_1_name']:
    t=t+1
    print(t,i)

1 Lin Shaye
2 Alex Pettyfer
3 Liam Neeson
4 Taraji P. Henson
5 Bruce Willis
6 Alex Lawther
7 Sam Hoffman 
8 Chris Hemsworth
9 Christian Gudegast 
10 Bethany Ashton Wolf 
11 Jesse Keller 
12 Dylan O'Brien
13 Dakota Fanning
14 Helen Mirren
15 Daniela Vega
16 Mario Van Peebles 
17 Gugu Mbatha-Raw
18 Bryan Coyne 
19 Rose Byrne
20 Akshay Kumar
21 Dakota Johnson
22 Anthony Sadler
23 Brian Crano 
24 Alex Ross Perry 
25 Chadwick Boseman
26 Jon Hamm
27 Taylor James
28 Jason Bateman
29 Alex Garland 
30 Angourie Rice
31 David Freyne 
32 Jennifer Lawrence
33 Bruce Willis
34 Logan Lerman
35 Usher Morgan 
36 Storm Reid
37 David Oyelowo
38 Cory Finley 
39 Christina Hendricks
40 Toby Kebbell
41 Alicia Vikander
42 Nick Robinson
43 J. Michael Finley
44 Rosamund Pike
45 Tessa Thompson
46 Sophie Turner
47 Zoey Deutch
48 John Boyega
49 Wes Anderson 
50 James McAvoy
51 Claire Foy
52 Andrew Hyatt 
53 Stanley Tucci 
54 Bella Thorne
55 Tye Sheridan
56 Tyler Perry 
57 Michael Mason 
58 Aaron Katz 
59 Adam Rifki

In [17]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    elif "(directors/screenplay)," in x:
        return ((x.split("(directors/screenplay), ")[-1]).split(", ")[1])
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [18]:
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x)).astype("str")

In [19]:
t=0
for i in df_2018['actor_2_name']:
    t=t+1
    print(t,i)

1 Angus Sampson
2 James Freedson-Jackson
3 Vera Farmiga
4 Jahi Di'Allo Winston
5 Cole Hauser
6 Abigail Breslin
7 Elliott Gould
8 Michael Shannon
9 Pablo Schreiber
10 Jessica Rothe
11 Molly Beucher
12 Kaya Scodelario
13 Toni Collette
14 Jason Clarke
15 Francisco Reyes Morandé
16 Ryan Guzman
17 Elizabeth Debicki
18 Graham Skipper
19 Domhnall Gleeson
20 Radhika Apte
21 Jamie Dornan
22 Alek Skarlatos
23 Dan Stevens
24 Adam Horowitz
25 Michael B. Jordan
26 Catherine Keener
27 Jackson Rathbone
28 Rachel McAdams
29 Jennifer Jason Leigh
30 Justice Smith
31 Sam Keeley
32 Joel Edgerton
33 Vincent D'Onofrio
34 Elle Fanning
35 Katie Vincent
36 Oprah Winfrey
37 Charlize Theron
38 Anya Taylor-Joy
39 Martin Henderson
40 Maggie Grace
41 Dominic West
42 Josh Duhamel
43 Madeline Carroll
44 Daniel Brühl
45 Melissa Leo
46 Dylan McDermott
47 Kathryn Hahn
48 Scott Eastwood
49 Koyu Rankin
50 Emily Blunt
51 Joshua Leonard
52 Olivier Martinez
53 Armie Hammer
54 Patrick Schwarzenegger
55 Olivia Cooke
56 Lyriq B

In [20]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [21]:
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))

In [22]:
t=0
for i in df_2018['actor_3_name']:
    t=t+1
    print(t,i)

1 Leigh Whannell
2 Emily Althaus
3 Patrick Wilson
4 Billy Brown
5 Shawn Ashmore
6 AnnaSophia Robb
7 Ingrid Michaelson
8 Michael Peña
9 O'Shea Jackson Jr.
10 John Benjamin Hickey
11 Connor Sullivan
12 Thomas Brodie-Sangster
13 Alice Eve
14 Sarah Snook
15 nan
16 Columbus Short
17 Aksel Hennie
18 Aly Fitzgerald
19 Sam Neill
20 nan
21 Eric Johnson
22 Spencer Stone
23 Gina Gershon
24 Mary-Louise Parker
25 Lupita Nyong'o
26 John Ortiz
27 Billy Zane
28 Billy Magnussen
29 Gina Rodriguez
30 Debby Ryan
31 Tom Vaughan-Lawlor
32 Matthias Schoenaerts
33 Elisabeth Shue
34 Michelle Monaghan
35 Joel Bernard
36 Reese Witherspoon
37 Joel Edgerton
38 Anton Yelchin
39 Bailee Madison
40 Ryan Kwanten
41 Walton Goggins
42 Jennifer Garner
43 Trace Adkins
44 Eddie Marsan
45 Whoopi Goldberg
46 Jack Kilmer
47 Tim Heidecker
48 Jing Tian
49 Edward Norton
50 Chiwetel Ejiofor
51 Jay Pharoah
52 James Faulkner
53 Clémence Poésy
54 Rob Riggle
55 Ben Mendelsohn
56 Crystle Stewart
57 Shane Harper
58 Greta Lee
59 Clark Du

In [23]:
df_2018

Title  \
0    Insidious: The Last Key   
1           The Strange Ones   
2               The Commuter   
3                 Proud Mary   
4           Acts of Violence   
..                       ...   
243               Second Act   
244          Holmes & Watson   
245                     Vice   
246      On the Basis of Sex   
247                Destroyer   

                                         Cast and crew  \
0    Adam Robitel (director); Leigh Whannell (scree...   
1    Christopher Radcliff (director/screenplay); La...   
2    Jaume Collet-Serra (director); Byron Willinger...   
3    Babak Najafi (director); John S. Newman, Chris...   
4    Brett Donowho (director); Nicolas Aaron Mezzan...   
..                                                 ...   
243  Peter Segal (director); Justin Zackham, Elaine...   
244  Etan Cohen (director/screenplay); Will Ferrell...   
245  Adam McKay (director/screenplay); Christian Ba...   
246  Mimi Leder (director); Daniel Stiepleman (scre...   
247  Karyn Kusama (director); Phil Hay, Matt Manfre...   

                      genres        id       director_name      actor_1_name  \
0            Horror Thriller  406563.0        Adam Robitel         Lin Shaye   
1              Drama Mystery  246252.0    Lauren Wolkstein     Alex Pettyfer   
2    Action Thriller Mystery  399035.0  Jaume Collet-Serra       Liam Neeson   
3      Thriller Action Crime  442064.0        Babak Najafi  Taraji P. Henson   
4      Action Crime Thriller  479040.0       Brett Donowho      Bruce Willis   
..                       ...       ...                 ...               ...   
243                      NaN       NaN         Peter Segal    Jennifer Lopez   
244                      NaN       NaN         Etan Cohen        Etan Cohen    
245                      NaN       NaN         Adam McKay        Adam McKay    
246                      NaN       NaN          Mimi Leder    Felicity Jones   
247                      NaN       NaN        Karyn Kusama     Nicole Kidman   

               actor_2_name     actor_3_name  
0             Angus Sampson   Leigh Whannell  
1    James Freedson-Jackson    Emily Althaus  
2              Vera Farmiga   Patrick Wilson  
3      Jahi Di'Allo Winston      Billy Brown  
4               Cole Hauser    Shawn Ashmore  
..                      ...              ...  
243             Leah Remini  Vanessa Hudgens  
244          John C. Reilly     Rebecca Hall  
245               Amy Adams     Steve Carell  
246            Armie Hammer   Justin Theroux  
247          Sebastian Stan     Toby Kebbell  

[248 rows x 8 columns]

In [24]:
df_2018 = df_2018.rename(columns={'Title':'movie_title'})

In [25]:
new_df18 = df_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title','id']]

In [26]:
new_df18

director_name      actor_1_name            actor_2_name  \
0          Adam Robitel         Lin Shaye           Angus Sampson   
1      Lauren Wolkstein     Alex Pettyfer  James Freedson-Jackson   
2    Jaume Collet-Serra       Liam Neeson            Vera Farmiga   
3          Babak Najafi  Taraji P. Henson    Jahi Di'Allo Winston   
4         Brett Donowho      Bruce Willis             Cole Hauser   
..                  ...               ...                     ...   
243         Peter Segal    Jennifer Lopez             Leah Remini   
244         Etan Cohen        Etan Cohen           John C. Reilly   
245         Adam McKay        Adam McKay                Amy Adams   
246          Mimi Leder    Felicity Jones            Armie Hammer   
247        Karyn Kusama     Nicole Kidman          Sebastian Stan   

        actor_3_name                   genres              movie_title  \
0     Leigh Whannell          Horror Thriller  Insidious: The Last Key   
1      Emily Althaus            Drama Mystery         The Strange Ones   
2     Patrick Wilson  Action Thriller Mystery             The Commuter   
3        Billy Brown    Thriller Action Crime               Proud Mary   
4      Shawn Ashmore    Action Crime Thriller         Acts of Violence   
..               ...                      ...                      ...   
243  Vanessa Hudgens                      NaN               Second Act   
244     Rebecca Hall                      NaN          Holmes & Watson   
245     Steve Carell                      NaN                     Vice   
246   Justin Theroux                      NaN      On the Basis of Sex   
247     Toby Kebbell                      NaN                Destroyer   

           id  
0    406563.0  
1    246252.0  
2    399035.0  
3    442064.0  
4    479040.0  
..        ...  
243       NaN  
244       NaN  
245       NaN  
246       NaN  
247       NaN  

[248 rows x 7 columns]

In [27]:
new_df18.isnull().sum()

director_name      0
actor_1_name       0
actor_2_name       0
actor_3_name      15
genres           107
movie_title        0
id               107
dtype: int64

In [28]:
new_df18=new_df18.dropna(how='any')

In [29]:
new_df18['director_name']=new_df18['director_name'].apply(lambda x:x.replace(" ",""))
new_df18['actor_1_name']=new_df18['actor_1_name'].apply(lambda x:x.replace(" ",""))
new_df18['actor_2_name']=new_df18['actor_2_name'].apply(lambda x:x.replace(" ",""))
new_df18['actor_3_name']=new_df18['actor_3_name'].apply(lambda x:x.replace(" ",""))

In [30]:
new_df18['tag'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']
new_df18['tag']=new_df18['tag'].apply(lambda x:x.lower())

In [31]:
new_df18=new_df18[["id","movie_title","tag"]]
new_df18

id              movie_title  \
0    406563.0  Insidious: The Last Key   
1    246252.0         The Strange Ones   
2    399035.0             The Commuter   
3    442064.0               Proud Mary   
4    479040.0         Acts of Violence   
..        ...                      ...   
136  492998.0                   Puzzle   
137  338676.0               Extinction   
138  420814.0        Christopher Robin   
139  445651.0        The Darkest Minds   
140  454992.0    The Spy Who Dumped Me   

                                                   tag  
0    linshaye angussampson leighwhannell adamrobite...  
1    alexpettyfer jamesfreedson-jackson emilyalthau...  
2    liamneeson verafarmiga patrickwilson jaumecoll...  
3    tarajip.henson jahidi'allowinston billybrown b...  
4    brucewillis colehauser shawnashmore brettdonow...  
..                                                 ...  
136  kellymacdonald irrfankhan daviddenman marcturt...  
137  michaelpeña lizzycaplan mikecolter benyoung ho...  
138  ewanmcgregor hayleyatwell jimcummings marcfors...  
139  amandlastenberg harrisdickinson mandymoore jen...  
140  milakunis katemckinnon justintheroux susannafo...  

[134 rows x 3 columns]

## Extracting features of 2019 movies from Wikipedia

In [33]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [34]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [35]:
df

Opening  Opening.1              Title  \
0      J A N U A R Y          4        Escape Room   
1      J A N U A R Y          4         Rust Creek   
2      J A N U A R Y          4   American Hangman   
3      J A N U A R Y         11   A Dog's Way Home   
4      J A N U A R Y         11         The Upside   
..               ...        ...                ...   
244  D E C E M B E R         25  Spies in Disguise   
245  D E C E M B E R         25       Little Women   
246  D E C E M B E R         25               1917   
247  D E C E M B E R         25         Just Mercy   
248  D E C E M B E R         27           Clemency   

                                    Production company  \
0                    Columbia Pictures / Original Film   
1                       IFC Films / Lunacy Productions   
2                          Hangman Justice Productions   
3                                    Columbia Pictures   
4                                    STX Entertainment   
..                                                 ...   
244  20th Century Fox Animation / Blue Sky Studios ...   
245            Columbia Pictures / Regency Enterprises   
246           Universal Pictures / DreamWorks Pictures   
247          Warner Bros. Pictures / Participant Media   
248                                               Neon   

                                         Cast and crew   Ref.  
0    Adam Robitel (director); Bragi F. Schut, Maria...    [2]  
1    Jen McGowan (director); Julie Lipson (screenpl...    [3]  
2    Wilson Coneybeare (director/screenplay); Donal...    [4]  
3    Charles Martin Smith (director); W. Bruce Came...    [5]  
4    Neil Burger (director); Jon Hartmere (screenpl...    [6]  
..                                                 ...    ...  
244  Nick Bruno, Troy Quane (directors); Brad Copel...  [132]  
245  Greta Gerwig (director/screenplay); Saoirse Ro...  [225]  
246  Sam Mendes (director/screenplay); Krysty Wilso...  [226]  
247  Destin Daniel Cretton (director/screenplay), A...  [227]  
248  Chinonye Chukwu (director/screenplay); Alfre W...  [228]  

[249 rows x 6 columns]

In [36]:
df['genres'] = df.progress_apply(lambda row: safe_get_genre(str(row['Title']), row.name), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 249/249 [06:40<00:00,  1.61s/it]


In [37]:
df['id'] = df.progress_apply(lambda row: safe_get_id(str(row['Title']), row.name), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 249/249 [00:00<00:00, 2560.57it/s]


In [38]:
df_2019=df[['Title','Cast and crew','genres','id']]

In [39]:
df_2019

Title                                      Cast and crew  \
0          Escape Room  Adam Robitel (director); Bragi F. Schut, Maria...   
1           Rust Creek  Jen McGowan (director); Julie Lipson (screenpl...   
2     American Hangman  Wilson Coneybeare (director/screenplay); Donal...   
3     A Dog's Way Home  Charles Martin Smith (director); W. Bruce Came...   
4           The Upside  Neil Burger (director); Jon Hartmere (screenpl...   
..                 ...                                                ...   
244  Spies in Disguise  Nick Bruno, Troy Quane (directors); Brad Copel...   
245       Little Women  Greta Gerwig (director/screenplay); Saoirse Ro...   
246               1917  Sam Mendes (director/screenplay); Krysty Wilso...   
247         Just Mercy  Destin Daniel Cretton (director/screenplay), A...   
248           Clemency  Chinonye Chukwu (director/screenplay); Alfre W...   

                                       genres      id  
0                     Horror Thriller Mystery  522681  
1                 Thriller Drama Action Crime  561362  
2                                    Thriller  567738  
3                      Drama Adventure Family  508763  
4                                Comedy Drama  440472  
..                                        ...     ...  
244  Animation Action Adventure Comedy Family  431693  
245                             Drama Romance  331482  
246                  War Action Drama History  530915  
247                       Drama Crime History  522212  
248                                     Drama  565307  

[249 rows x 4 columns]

In [40]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    elif " (directors/screenplay);" in x:
        return x.split("(directors/screenplay); ")[0]
    elif " (director, screenplay);" in x:
        return  x.split("(director, screenplay); ")[0]
    elif "(director/screenplay);" in x:
        return x.split("(director/screenplay);")[0] 
    elif "(director/screenplay)," in x:
        return x.split("(director/screenplay),")[0] 
    else:
        return  x.split("(screenplay);")[0]

In [41]:
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))

In [42]:
t=0
for i in df_2019['director_name']:
    t=t+1
    print(t-1,i)

0 Adam Robitel
1 Jen McGowan
2 Wilson Coneybeare 
3 Charles Martin Smith
4 Neil Burger
5 Jeffrey Nachmanoff
6 Batán Silva
7 M. Night Shyamalan 
8 Vicky Jewson
9 Henry Dunham 
10 Timothy Woodward Jr.
11 Joe Cornish 
12 Steven Knight 
13 Grant Sputore
14 Andrew Patterson
15 Catherine Hardwicke
16 Dan Gilroy 
17 Madeleine Sackler
18 Mike Mitchell
19 Adam Shankman
20 Hans Petter Moland
21 Steven Soderbergh
22 Nicholas McCarthy
23 Todd Strauss-Schulson
24 Christopher Landon 
25 Robert Rodriguez
26 Dean DeBlois 
27 Stephen Merchant 
28 Tyler Perry
29 Neil Jordan 
30 J. C. Chandor 
31 Anna Boden, Ryan Fleck 
32 Sebastián Lelio 
33 Vincent D'Onofrio
34 Josh Appelbaum, Andre Nemec 
35 Katt Shea
36 Rupert Wyatt 
37 James Kent
38 Justin Baldoni
39 John Lee Hancock
40 Robert Rodriguez 
41 Ivan Kavanagh 
42 Jesse V. Johnson
43 Jordan Peele 
44 Anthony Maras 
45 Jeff Tremaine
46 S. Craig Zahler 
47 Tim Burton
48 Harmony Korine 
49 Chuck Konzelman, Cary Soloman 
50 David F. Sandberg
51 Kevin Kolsch, 

In [43]:
def get_actor1(x):
    if " (screenplay)" in x:
        return ((x.split("screenplay); ")[-1]).split(", ")[0])
    elif "(co-director);" in x :
        return ((x.split("(co-director); ")[-1]).split(", ")[0]) 
    
    
    elif "(director/screenplay);" in x:
        return ((x.split('(director/screenplay); ')[-1]).split(",")[0])
    
    
    elif "(writers);" in x:
        return ((x.split('(writers); ')[-1]).split(",")[0]) 
    
    elif " (screenplay/story)," in x:
        return ((x.split('(screenplay/story), ')[-1]).split(",")[0])
    
    elif "(co-director);" in x :
        return ((x.split("(co-director); ")[-1]).split(", ")[0]) 
    elif " (directors/screenplay);" in x:
        return ((x.split("(directors/screenplay); ")[-1]).split(", ")[0]) 
    
    else:
        return np.NaN#((x.split('(director); ')[-1]).split(",")[0])

In [44]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))

In [45]:
t=0
for i in df_2019['actor_1_name']:
    t=t+1
    print(t-1,i)

0 Taylor Russell
1 Hermione Corfield
2 Donald Sutherland
3 Bryce Dallas Howard
4 Bryan Cranston
5 Keanu Reeves
6 Natalia Dyer
7 James McAvoy
8 Noomi Rapace
9 James Badge Dale
10 Lin Shaye
11 Louis Ashbourne Serkis
12 Matthew McConaughey
13 Clara Rugaard
14 Sierra McCormick
15 Gina Rodriguez
16 Jake Gyllenhaal
17 Jeffrey Wright
18 Chris Pratt
19 Taraji P. Henson
20 Liam Neeson
21 André Holland
22 Taylor Schilling
23 Rebel Wilson
24 Jessica Rothe
25 Rosa Salazar
26 Jay Baruchel
27 Florence Pugh
28 Tyler Perry
29 Isabelle Huppert
30 Ben Affleck
31 Brie Larson
32 Julianne Moore
33 Ethan Hawke
34 Brianna Denski
35 Sophia Lillis
36 John Goodman
37 Alexander Skarsgård
38 Cole Sprouse
39 Kevin Costner
40 Roby Attal
41 Emile Hirsch
42 Tony Jaa
43 Lupita Nyong'o
44 Dev Patel
45 Douglas Booth
46 Mel Gibson
47 Colin Farrell
48 Matthew McConaughey
49 Ashley Bratcher
50 Zachary Levi
51 Jason Clarke
52 Taraji P. Henson
53 Robert Pattinson
54 Brie Larson
55 Hilary Duff
56 Caitlin Gerard
57 Ashton Sand

In [46]:
def get_actor2(x):
    if "(story);" in x:
        return ((x.split("(story);")[-1]).split(", ")[1])
    
    elif len((x.split("(screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    elif "(screenplay); " in x:
        return ((x.split("(screenplay); ")[-1]).split(", ")[1])
    
    elif "(director/screenplay);"in x:
        return ((x.split("(director/screenplay); ")[-1]).split(", ")[1])
    
    elif "(co-director);"in x:
        if len((x.split("(co-director); ")[-1]).split(", ")) < 2:
            return np.NaN
        else:
            return ((x.split("(co-director); ")[-1]).split(", ")[1])
    elif "(director, screenplay);" in x:
        return ((x.split("(director, screenplay); ")[-1]).split(", ")[1])
        
    elif " (directors/screenplay);" in x:
        return ((x.split("(directors/screenplay); ")[-1]).split(", ")[1])
        
    else:
        return ((x.split("(screenplay/story), ")[-1]).split(", ")[1])

In [47]:
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))

In [48]:
t=0
for i in df_2019['actor_2_name']:
    t=t+1
    print(t-1,i)

0 Logan Miller
1 Jay Paulson
2 Vincent Kartheiser
3 Edward James Olmos
4 Kevin Hart
5 nan
6 Kyra Sedgwick
7 Bruce Willis
8 Sophie Nélisse
9 Brian Geraghty
10 Michael Welch
11 Dean Chaumoo
12 Anne Hathaway
13 Rose Byrne
14 James Montague
15 Ismael Cruz Córdova
16 Rene Russo
17 William Fichtner
18 Elizabeth Banks
19 Tracy Morgan
20 Tom Bateman
21 Zazie Beetz
22 Peter Mooney
23 Liam Hemsworth
24 Israel Broussard
25 Christoph Waltz
26 America Ferrera
27 Lena Headey
28 Courtney Burrell
29 Chloë Grace Moretz
30 Oscar Isaac
31 Samuel L. Jackson
32 John Turturro
33 Dane DeHaan
34 Ken Hudson Campbell
35 Zoe Renee
36 Ashton Sanders
37 Keira Knightley
38 Haley Lu Richardson
39 Woody Harrelson
40 Lauren Hatfield
41 Tim Ahern
42 Iko Uwais
43 Winston Duke
44 Armie Hammer
45 Iwan Rheon
46 Vince Vaughn
47 Michael Keaton
48 Zac Efron
49 Brooks Ryan
50 Asher Angel
51 Amy Seimetz
52 Sam Rockwell
53 Juliette Binoche
54 Samuel L. Jackson
55 Jonathan Bennett
56 Ashley Zukerman
57 Margaret Qualley
58 Stanley

In [49]:
def get_actor3(x):
    if " (screenplay);" in x:
        if len((x.split("(screenplay); ")[-1]).split(", ")) < 3:
            return np.NaN
        else:
            return ((x.split("(screenplay); ")[-1]).split(", ")[2])
    else:
        if len((x.split("(writers); ")[-1]).split(", ")) < 3:
            return np.NaN
        else:
            return ((x.split("(writers); ")[-1]).split(", ")[2])

In [50]:
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [51]:
t=0
for i in df_2019['actor_3_name']:
    t=t+1
    print(t-1,i)

0 Deborah Ann Woll
1 Sean O'Bryan
2 Oliver Dennis
3 Alexandra Shipp
4 Nicole Kidman
5 nan
6 Tim Daly
7 Samuel L. Jackson
8 Indira Varma
9 nan
10 Melissa Bolona
11 Tom Taylor
12 Jason Clarke
13 Clara Rugaard
14 nan
15 Anthony Mackie
16 Toni Collette
17 Boyd Holbrook
18 Tiffany Haddish
19 Max Greenfield
20 Tom Jackson
21 Melvin Gregg
22 Colm Feore
23 Adam DeVine
24 Phi Vu
25 Jennifer Connelly
26 Cate Blanchett
27 Nick Frost
28 Patrice Lovely
29 Maika Monroe
30 Charlie Hunnam
31 Ben Mendelsohn
32 Michael Cera
33 Jake Schur
34 Matthew Broderick
35 Mackenzie Graham
36 Jonathan Majors
37 Jason Clarke
38 nan
39 Kathy Bates
40 Alejandro Rose Garcia
41 Déborah François
42 Tiger Chen
43 Shahadi Wright Joseph
44 Nazanin Boniadi
45 Machine Gun Kelly
46 Michael Jai White
47 Danny DeVito
48 Jonah Hill
49 Brooks Ryan
50 Mark Strong
51 John Lithgow
52 nan
53 André Benjamin
54 Joan Cusack
55 Lydia Hearst
56 Julia Goldani Telles
57 Nick Robinson
58 Miranda Otto
59 Ian McShane
60 Marsai Martin
61 Zoe Sal

In [52]:
df_2019 = df_2019.rename(columns={'Title':'movie_title'})

In [53]:
df_2019

movie_title                                      Cast and crew  \
0          Escape Room  Adam Robitel (director); Bragi F. Schut, Maria...   
1           Rust Creek  Jen McGowan (director); Julie Lipson (screenpl...   
2     American Hangman  Wilson Coneybeare (director/screenplay); Donal...   
3     A Dog's Way Home  Charles Martin Smith (director); W. Bruce Came...   
4           The Upside  Neil Burger (director); Jon Hartmere (screenpl...   
..                 ...                                                ...   
244  Spies in Disguise  Nick Bruno, Troy Quane (directors); Brad Copel...   
245       Little Women  Greta Gerwig (director/screenplay); Saoirse Ro...   
246               1917  Sam Mendes (director/screenplay); Krysty Wilso...   
247         Just Mercy  Destin Daniel Cretton (director/screenplay), A...   
248           Clemency  Chinonye Chukwu (director/screenplay); Alfre W...   

                                       genres      id           director_name  \
0                     Horror Thriller Mystery  522681            Adam Robitel   
1                 Thriller Drama Action Crime  561362             Jen McGowan   
2                                    Thriller  567738      Wilson Coneybeare    
3                      Drama Adventure Family  508763    Charles Martin Smith   
4                                Comedy Drama  440472             Neil Burger   
..                                        ...     ...                     ...   
244  Animation Action Adventure Comedy Family  431693  Nick Bruno, Troy Quane   
245                             Drama Romance  331482           Greta Gerwig    
246                  War Action Drama History  530915             Sam Mendes    
247                       Drama Crime History  522212  Destin Daniel Cretton    
248                                     Drama  565307        Chinonye Chukwu    

            actor_1_name          actor_2_name      actor_3_name  
0         Taylor Russell          Logan Miller  Deborah Ann Woll  
1      Hermione Corfield           Jay Paulson      Sean O'Bryan  
2      Donald Sutherland    Vincent Kartheiser     Oliver Dennis  
3    Bryce Dallas Howard    Edward James Olmos   Alexandra Shipp  
4         Bryan Cranston            Kevin Hart     Nicole Kidman  
..                   ...                   ...               ...  
244           Will Smith           Tom Holland     Rashida Jones  
245        Saoirse Ronan           Emma Watson     Florence Pugh  
246        George MacKay  Dean-Charles Chapman       Mark Strong  
247    Michael B. Jordan            Jamie Foxx       Brie Larson  
248        Alfre Woodard        Wendell Pierce       Aldis Hodge  

[249 rows x 8 columns]

In [54]:
df_2019.isnull().sum()

movie_title       0
Cast and crew     0
genres            0
id                0
director_name     0
actor_1_name      2
actor_2_name      2
actor_3_name     18
dtype: int64

In [55]:
df_2019 = df_2019.dropna(how='any')

In [56]:
new_df19 = df_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title','id']]

In [57]:
new_df19['director_name']=new_df19['director_name'].apply(lambda x:x.replace(" ",""))
new_df19['actor_1_name']=new_df19['actor_1_name'].apply(lambda x:x.replace(" ",""))
new_df19['actor_2_name']=new_df19['actor_2_name'].apply(lambda x:x.replace(" ",""))
new_df19['actor_3_name']=new_df19['actor_3_name'].apply(lambda x:x.replace(" ",""))

In [58]:
new_df19['tag'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' '+ new_df19['actor_3_name'] + ' '+ new_df19['director_name'] +' ' + new_df19['genres']
new_df19['tag']=new_df19['tag'].apply(lambda x:x.lower())

In [59]:
new_df19=new_df19[["id","movie_title","tag"]]
new_df19

id        movie_title  \
0    522681        Escape Room   
1    561362         Rust Creek   
2    567738   American Hangman   
3    508763   A Dog's Way Home   
4    440472         The Upside   
..      ...                ...   
244  431693  Spies in Disguise   
245  331482       Little Women   
246  530915               1917   
247  522212         Just Mercy   
248  565307           Clemency   

                                                   tag  
0    taylorrussell loganmiller deborahannwoll adamr...  
1    hermionecorfield jaypaulson seano'bryan jenmcg...  
2    donaldsutherland vincentkartheiser oliverdenni...  
3    brycedallashoward edwardjamesolmos alexandrash...  
4    bryancranston kevinhart nicolekidman neilburge...  
..                                                 ...  
244  willsmith tomholland rashidajones nickbruno,tr...  
245  saoirseronan emmawatson florencepugh gretagerw...  
246  georgemackay dean-charleschapman markstrong sa...  
247  michaelb.jordan jamiefoxx brielarson destindan...  
248  alfrewoodard wendellpierce aldishodge chinonye...  

[230 rows x 3 columns]

In [60]:
my_df = pd.concat([new_df18, new_df19], ignore_index=True)

In [61]:
my_df

id              movie_title  \
0    406563.0  Insidious: The Last Key   
1    246252.0         The Strange Ones   
2    399035.0             The Commuter   
3    442064.0               Proud Mary   
4    479040.0         Acts of Violence   
..        ...                      ...   
359  431693.0        Spies in Disguise   
360  331482.0             Little Women   
361  530915.0                     1917   
362  522212.0               Just Mercy   
363  565307.0                 Clemency   

                                                   tag  
0    linshaye angussampson leighwhannell adamrobite...  
1    alexpettyfer jamesfreedson-jackson emilyalthau...  
2    liamneeson verafarmiga patrickwilson jaumecoll...  
3    tarajip.henson jahidi'allowinston billybrown b...  
4    brucewillis colehauser shawnashmore brettdonow...  
..                                                 ...  
359  willsmith tomholland rashidajones nickbruno,tr...  
360  saoirseronan emmawatson florencepugh gretagerw...  
361  georgemackay dean-charleschapman markstrong sa...  
362  michaelb.jordan jamiefoxx brielarson destindan...  
363  alfrewoodard wendellpierce aldishodge chinonye...  

[364 rows x 3 columns]

In [62]:
old_df = pd.read_csv('till 2017.csv')

In [63]:
old_df

movie_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
5227              The Incredible Jessica James   
5228                         Can't Buy My Love   
5229          Thick Lashes of Lauri Mäntyvaara   
5230               Cop and a Half: New Recruit   
5231                                       Mom   

                                                    tag      id  
0     cchpounder joeldavidmoore wesstudi jamescamero...   19995  
1     johnnydepp orlandobloom jackdavenport goreverb...     285  
2     christophwaltz rorykinnear stephaniesigman sam...  206647  
3     tomhardy christianbale josephgordon-levitt chr...   49026  
4     darylsabara samanthamorton pollywalker andrews...   49529  
...                                                 ...     ...  
5227  jessicawilliams chriso'dowd keithstanfield jim...  432789  
5228  adelaidekane benjaminhollingsworth jeanlouisak...  445126  
5229  inkahaapamäki rosahonkonen tiitusrantala hanna...  468707  
5230  loudiamondphillips wallaceshawn ginaholden jon...  461297  
5231  sridevikapoor sajalali akshayekhanna raviudyaw...  404604  

[5232 rows x 3 columns]

In [64]:
final_df = pd.concat([old_df, my_df], ignore_index=True)

In [65]:
final_df

movie_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
5591                         Spies in Disguise   
5592                              Little Women   
5593                                      1917   
5594                                Just Mercy   
5595                                  Clemency   

                                                    tag        id  
0     cchpounder joeldavidmoore wesstudi jamescamero...   19995.0  
1     johnnydepp orlandobloom jackdavenport goreverb...     285.0  
2     christophwaltz rorykinnear stephaniesigman sam...  206647.0  
3     tomhardy christianbale josephgordon-levitt chr...   49026.0  
4     darylsabara samanthamorton pollywalker andrews...   49529.0  
...                                                 ...       ...  
5591  willsmith tomholland rashidajones nickbruno,tr...  431693.0  
5592  saoirseronan emmawatson florencepugh gretagerw...  331482.0  
5593  georgemackay dean-charleschapman markstrong sa...  530915.0  
5594  michaelb.jordan jamiefoxx brielarson destindan...  522212.0  
5595  alfrewoodard wendellpierce aldishodge chinonye...  565307.0  

[5596 rows x 3 columns]

In [66]:
final_df.isnull().sum()

movie_title    0
tag            0
id             0
dtype: int64

In [67]:
final_df.to_csv('till 2019.csv',index=False)